In [1]:
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt

In [5]:
def nothing(x):
    pass
 
cap= cv2.VideoCapture(0,cv2.CAP_DSHOW) 
cap.set(3,1280) 
cap.set(4,720)

cv2.namedWindow("Trackbars")
 
cv2.createTrackbar("L - H", "Trackbars", 0, 179, nothing)
cv2.createTrackbar("L - S", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("L - V", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("U - H", "Trackbars", 179, 179, nothing)
cv2.createTrackbar("U - S", "Trackbars", 255, 255, nothing)
cv2.createTrackbar("U - V", "Trackbars", 255, 255, nothing)
 
 
while True:
    _, frame = cap.read()
    frame = cv2.flip( frame, 1 ) 

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
 
    l_h = cv2.getTrackbarPos("L - H", "Trackbars")
    l_s = cv2.getTrackbarPos("L - S", "Trackbars")
    l_v = cv2.getTrackbarPos("L - V", "Trackbars")
    u_h = cv2.getTrackbarPos("U - H", "Trackbars")
    u_s = cv2.getTrackbarPos("U - S", "Trackbars")
    u_v = cv2.getTrackbarPos("U - V", "Trackbars")
 
    # set the lower and upper range according to the value selected by the trackbar.
    lower_range = np.array([l_h, l_s, l_v])
    upper_range = np.array([u_h, u_s, u_v])
    
    # filter and get the binary mask, where white represents your target color.
    mask = cv2.inRange(hsv, lower_range, upper_range)
 
    # optionally you can also show the real part of the target color
    res = cv2.bitwise_and(frame, frame, mask=mask)
    
    mask_3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    
    # stack all frames and show it
    stacked = np.hstack((mask_3,frame,res))
    cv2.imshow('Trackbars',cv2.resize(stacked,None,fx=0.4,fy=0.4))
 
    key = cv2.waitKey(1)
    if key == 27:
        break
        
    if key == ord('s'):
        thearray = [[l_h,l_s,l_v],[u_h, u_s, u_v]]
        print(thearray)
        
        # Also save this array as harryval.npy
        np.save('harryval',thearray)
        break
    
cap.release()
cv2.destroyAllWindows()

[[0, 85, 130], [179, 255, 255]]


In [9]:
useload =True

if useload:
    harryval = np.load('harryval.npy')

noise_thresh = 4000

# We will give the user a couple of seconds to get away from the camera in order to save the background
time.sleep(3)  

cap = cv2.VideoCapture(0,cv2.CAP_DSHOW)
cap.set(3,1280)
cap.set(4,720)
# just disabling the autofoucs in my usb cam.
cap.set(cv2.CAP_PROP_AUTOFOCUS, 0)

### Now this piece of code below just gives us more robust background picture

# What we're doing here is that we are averaging around a number of frames instead of just taking a single picture as background
# Also notice we are ignoring the first 50 frames, because a lot of times there is noise in these initial frames.
freshup=80
frames=[]
for i in range(freshup):  
    ret ,frame = cap.read() 
    if i > 50:        
        frames.append(frame.astype('float64'))
        
sumframes= sum(frames) / len(frames)
sumframes = sumframes.astype('uint8')

# Save the final average frame as background, also flip it as we're doing it in the loop too.
background = cv2.flip( sumframes, 1 )    


cv2.namedWindow('Result', cv2.WINDOW_NORMAL)

while(1):
   # try:
        # Take each frame
        _, frame = cap.read()
        frame = cv2.flip( frame, 1 )
        
        kernel = np.ones((5,5),np.uint8)
        
        # Convert BGR to HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # If you're reading from memory then load the upper and lower ranges from there
        if useload:
                lower_range = harryval[0]
                upper_range = harryval[1]

        # Otherwise define your own custom values for upper and lower range.
        else:             
           lower_range  = np.array([26,80,147])
           upper_range = np.array([81,255,255])

        mask = cv2.inRange(hsv, lower_range, upper_range)

        # perform the morphological operations to get rid of the noise
        mask = cv2.erode(mask,kernel,iterations = 1)
        mask = cv2.dilate(mask,kernel,iterations = 1)


        contours,hierarchy = cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        
        # Make sure you have detected some contours and its not noise
        if len(contours) > 0 and cv2.contourArea(max(contours, key = cv2.contourArea)) > noise_thresh:
                   c = max(contours, key = cv2.contourArea)               

                   hull = cv2.convexHull(c)
                    
                   # Optionally you can draw the hull for visualizing
                   #cv2.drawContours(frame, [hull], -1, (0,0,220), 3)
                    
                   mask_blanket_only = np.zeros(frame.shape[:2],np.uint8)
                   
                   # We are dialting the blanket only after extracting its contour.
                   cv2.drawContours(mask_blanket_only,[hull],0,255,-1)
                   #cv2.drawContours(mask_blanket_only,[c],0,255,-1)

                   mask_blanket_only = cv2.dilate(mask_blanket_only,kernel,iterations = 3)

                   img_fg = cv2.bitwise_and(background,background, mask = mask_blanket_only)
                   img_bg = cv2.bitwise_and(frame,frame, mask = cv2.bitwise_not(mask_blanket_only))

                   combined = cv2.add(img_bg,img_fg)
                   cv2.imshow('Result',combined)
                   
                    #You can visualize the mask for debugging
                   #cv2.imshow('mask2',mask_blanket_only)

                
        # if there is no blanket detected then show the original blanket.
        else:
         cv2.imshow('Result',frame)
        
        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            break


cv2.destroyAllWindows()
cap.release()